In [331]:

import sys
import os
from PIL import Image
import torchvision.utils as vutils

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

from models import Generator, Discriminator, FeatureExtractor


try:
    os.makedirs('output/high_res_fake')
    os.makedirs('output/high_res_real')
    os.makedirs('output/low_res')
except OSError:
    pass

imageSize = 15
upSampling = 2

transform = transforms.Compose([#transforms.RandomCrop(imageSize*upSampling),
                                #transforms.FiveCrop(15),
                                transforms.ToTensor()])

normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                std = [0.229, 0.224, 0.225])

scale = transforms.Compose([transforms.ToPILImage(),
#                             transforms.Resize(imageSize),
                            transforms.ToTensor(),
                            transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                                std = [0.229, 0.224, 0.225])
                            ])

# Equivalent to un-normalizing ImageNet (for correct visualization)
unnormalize = transforms.Normalize(mean = [-2.118, -2.036, -1.804], std = [4.367, 4.464, 4.444])
unscale = transforms.Compose([
                            transforms.ToPILImage()
                            
                            ])


In [353]:
# img = Image.open("../DCgan/DCGANresults/fake_samples_epoch_079.png")
img = Image.open("../SN-GAN/log/fake_samples_epoch_005.png")
img = img.crop((35, 35, 48, 48))
generator = Generator(16, upSampling)
generator.load_state_dict(torch.load('checkpoints/generator_final.pth'))

img

In [354]:
def convert_img(img_tensor, nrow):

    img_tensor = unnormalize(img_tensor)
    im = unscale(img_tensor)

    return im

In [355]:

x = scale(transform(img))
low_res = Variable(scale(transform(img))).unsqueeze(0)

low_im = convert_img(x.data, 1)

low_im


In [356]:
high_res_fake = generator(Variable(low_res)).squeeze()

# unscale(high_res_fake)

im = convert_img(high_res_fake.data, 1)

im